<font color='green'>***Installation and Libraries Import***</font>
---
--- 

In [224]:

# %pip install flwr
# %pip install ray 
# %pip install --upgrade pip
# %pip install torch torchvision matplotlib
# %pip install async-timeout
# %pip install async-numpy
# %pip install pandas
# %pip install datasets
# %pip install scikit-learn


In [225]:
import os
import flwr as fl
import torch, ray, pandas, sklearn
import pickle
from pprint import pprint


# print("All modules loaded successfully!")
# print("FLWR version:", fl.__version__)
# print("Ray version:", ray.__version__)
# print("Torch version:", torch.__version__)

# libraries_to_uninstall = [
#     "tb-nightly==2.18.0a20240701",
#     "tensorboard==2.16.2",
#     "tensorboard-data-server==0.7.2",
#     "tensorboard-plugin-wit==1.8.1",
#     "tensorflow==2.16.2",
#     "tensorflow-io-gcs-filesystem==0.37.0",
#     "termcolor==2.4.0",
#     "terminado==0.18.1",
#     "tf-estimator-nightly==2.8.0.dev2021122109",
#     "tf_keras-nightly==2.18.0.dev2024070109",
#     "tf-nightly==2.18.0.dev20240626"
# ]
# for library in libraries_to_uninstall:
#     os.system(f"pip uninstall -y {library}")
# print("All modules loaded successfully!")
# print("FLWR version:", fl.__version__)
# print("Ray version:", ray.__version__)
# print("Torch version:", torch.__version__)

In [226]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader, random_split, Subset
import torch.optim as optim
from torch.utils.data import Dataset, WeightedRandomSampler
import flwr as fl
from flwr.common import Metrics
# from flwr_datasets import FederatedDataset
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
import random
import math
import pickle
import csv
import copy
print(fl.__version__)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

1.5.0
cpu


<font color='Brown'>***Constants***</font>
---
--- 

In [227]:
# DEVICE
NUM_CLIENTS = 48 #48
ROUNDS = 40 #40
BATCH_SIZE = 64 #C
LEARNING_RATE = 0.0025 #C
EPOCHS = 3 #C
DATA_GROUPS = 120
BATCH_ROUND = 40
SIZE_ROUND = int(BATCH_ROUND * BATCH_SIZE * NUM_CLIENTS)
NUM_ATCKS = 11
FL = True
PATH = f'DNNCent{NUM_CLIENTS}-clients-{NUM_ATCKS}atk-{ROUNDS}-rounds-{EPOCHS}-epochs-{LEARNING_RATE}-lr-{DATA_GROUPS}-groups'
G = 0

<font color='Light Blue'>***Dataset Preparations***</font>
---
--- 

In [228]:
TrafficData = {}
TrafficData['Dataset']={}
sets_names = ['30','100','70','50','120']
for  DATA_NUM in sets_names:
    TrafficData['Dataset'][DATA_NUM]=pd.read_csv(f'data/2_Dataset_{NUM_ATCKS}_Attack_{DATA_NUM}_normal.csv', low_memory=False, quoting=csv.QUOTE_NONE, on_bad_lines='skip')
    print(DATA_NUM, TrafficData['Dataset'][DATA_NUM].shape)
for DATA_NUM in TrafficData['Dataset']:
    TrafficData['Dataset'][DATA_NUM]=TrafficData['Dataset'][DATA_NUM].sample(frac=1, random_state=42).reset_index(drop=True)

30 (184320, 99)
100 (184320, 99)
70 (184320, 99)
50 (184320, 99)
120 (120000, 99)


In [229]:
TrafficData['Split'] = {}
sets_training =  ['30','100','70','50']
for DATA_NUM in sets_training:
    TrafficData['Split'][DATA_NUM] = np.array_split(TrafficData['Dataset'][DATA_NUM],DATA_GROUPS)

TrafficData['Combined'] = pd.concat([TrafficData['Split']['30'][0], TrafficData['Split']['100'][0], TrafficData['Split']['70'][0], TrafficData['Split']['50'][0]]).reset_index(drop=True)
for GROUP in range(1, DATA_GROUPS):
    TrafficData['Combined'] = pd.concat([TrafficData['Combined'], TrafficData['Split']['30'][GROUP], TrafficData['Split']['100'][GROUP], TrafficData['Split']['70'][GROUP], TrafficData['Split']['50'][GROUP]]).reset_index(drop=True)
print(TrafficData['Combined'].shape)            

/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


(737280, 99)


In [230]:
TrafficData['Train'] = {}
TrafficData['Train']['X'] = TrafficData['Combined'].iloc[:, 0:-1]
TrafficData['Train']['y'] = TrafficData['Combined'].iloc[:, -1]
print(TrafficData['Train']['X'].shape)
print(TrafficData['Train']['y'].shape)

TrafficData['Test'] = {}
TrafficData['Test']['X']=TrafficData['Dataset']['120'].iloc[:, 0:-1]
TrafficData['Test']['y']=TrafficData['Dataset']['120'].iloc[:, -1]
print(TrafficData['Test']['X'].shape)
print(TrafficData['Test']['y'].shape)

(737280, 98)
(737280,)
(120000, 98)
(120000,)


In [231]:
scaler = MinMaxScaler()
model = scaler.fit(TrafficData['Train']['X'])
TrafficData['Train']['X'] = model.transform(TrafficData['Train']['X'])
TrafficData['Test']['X'] = model.transform(TrafficData['Test']['X'])

TrafficData['Train']['X'], TrafficData['Train']['y']= np.array(TrafficData['Train']['X']), np.array(TrafficData['Train']['y'])
print(type(TrafficData['Train']['X']),type(TrafficData['Train']['y']))
print(TrafficData['Train']['X'].shape,TrafficData['Train']['y'].shape)
TrafficData['Test']['X'], TrafficData['Test']['y']= np.array(TrafficData['Test']['X']), np.array(TrafficData['Test']['y'])
print(type(TrafficData['Test']['X']),type(TrafficData['Test']['y']))
print(TrafficData['Test']['X'].shape,TrafficData['Test']['y'].shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(737280, 98) (737280,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(120000, 98) (120000,)


In [232]:
TrafficData['ROUNDS']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['ROUNDS'][ROUND]={}

SIZE_Demo = SIZE_ROUND
for ROUND in range(1,ROUNDS+1):
    if ROUND == 1:
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][:SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][:SIZE_Demo]
    else:
        print((SIZE_Demo - SIZE_ROUND),SIZE_Demo)
        TrafficData['ROUNDS'][ROUND]['X']= TrafficData['Train']['X'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
        TrafficData['ROUNDS'][ROUND]['y']= TrafficData['Train']['y'][(SIZE_Demo - SIZE_ROUND):SIZE_Demo]
    SIZE_Demo = SIZE_Demo + SIZE_ROUND
for ROUND in TrafficData['ROUNDS']:
    print("ROUND: ", ROUND, TrafficData['ROUNDS'][ROUND]['X'].shape, TrafficData['ROUNDS'][ROUND]['y'].shape)
print(SIZE_Demo, SIZE_ROUND)

122880 245760
245760 368640
368640 491520
491520 614400
614400 737280
737280 860160
860160 983040
983040 1105920
1105920 1228800
1228800 1351680
1351680 1474560
1474560 1597440
1597440 1720320
1720320 1843200
1843200 1966080
1966080 2088960
2088960 2211840
2211840 2334720
2334720 2457600
2457600 2580480
2580480 2703360
2703360 2826240
2826240 2949120
2949120 3072000
3072000 3194880
3194880 3317760
3317760 3440640
3440640 3563520
3563520 3686400
3686400 3809280
3809280 3932160
3932160 4055040
4055040 4177920
4177920 4300800
4300800 4423680
4423680 4546560
4546560 4669440
4669440 4792320
4792320 4915200
ROUND:  1 (122880, 98) (122880,)
ROUND:  2 (122880, 98) (122880,)
ROUND:  3 (122880, 98) (122880,)
ROUND:  4 (122880, 98) (122880,)
ROUND:  5 (122880, 98) (122880,)
ROUND:  6 (122880, 98) (122880,)
ROUND:  7 (0, 98) (0,)
ROUND:  8 (0, 98) (0,)
ROUND:  9 (0, 98) (0,)
ROUND:  10 (0, 98) (0,)
ROUND:  11 (0, 98) (0,)
ROUND:  12 (0, 98) (0,)
ROUND:  13 (0, 98) (0,)
ROUND:  14 (0, 98) (0,)
ROUN

In [233]:
class ClassifierDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__ (self):
        return len(self.X_data)

In [234]:
TrafficData['trainsets']={}
for ROUND in range(1, ROUNDS+1):
    TrafficData['trainsets'][ROUND]= ClassifierDataset(torch.from_numpy(TrafficData['ROUNDS'][ROUND]['X']).float(), torch.from_numpy(TrafficData['ROUNDS'][ROUND]['y']).long())
TrafficData['testset'] = ClassifierDataset(torch.from_numpy(TrafficData['Test']['X']).float(), torch.from_numpy(TrafficData['Test']['y']).long())

In [235]:
def load_train(numberofclients, ROUND):    
    # Use the actual dataset length to compute balanced portions per client
    dataset_len = len(TrafficData['trainsets'][ROUND])
    if dataset_len == 0:
        # Return an empty list of loaders to avoid indexing errors downstream
        return [DataLoader(Subset(TrafficData['trainsets'][ROUND], []), batch_size=BATCH_SIZE, shuffle=False) for _ in range(numberofclients)]
    
    # Distribute samples as evenly as possible across clients (handle remainders)
    num_portions = int(numberofclients)
    base_portion = dataset_len // num_portions
    remainder = dataset_len % num_portions
    portion_indices = []
    start_idx = 0
    for i in range(num_portions):
        # First `remainder` clients receive one extra sample
        sz = base_portion + (1 if i < remainder else 0)
        end_idx = start_idx + sz
        if sz > 0:
            portion_indices.append(list(range(start_idx, end_idx)))
        else:
            portion_indices.append([])
        start_idx = end_idx
    
    # Build Subset and DataLoader for each client (safe: indices are within [0, dataset_len))
    portion_datasets = [Subset(TrafficData['trainsets'][ROUND], indices) for indices in portion_indices]
    portion_loaders = [DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False) for dataset in portion_datasets]            
    return portion_loaders
def load_test(numberofclients):    
    testloader = DataLoader(TrafficData['testset'], batch_size=BATCH_SIZE, shuffle=False)
    return testloader

In [236]:
Dataloaders = {}
for ROUND in range(1, ROUNDS+1):
    Dataloaders[ROUND] = load_train(NUM_CLIENTS, ROUND)
Dataloaders['Test'] = load_test(NUM_CLIENTS)

In [237]:
# Diagnostic: print per-client dataset sizes for the first few rounds to verify splits
for ROUND in range(1, min(6, ROUNDS+1)):
    loaders = Dataloaders.get(ROUND, None)
    if loaders is None:
        print(f"Dataloaders for Round {ROUND} not found (run the rebuild cell).")
        continue
    sizes = [len(loader.dataset) for loader in loaders]
    print(f"Round {ROUND}: per-client sizes (first 12): {sizes[:12]}")
    print(f"  Total samples this round: {sum(sizes)}\n")

Round 1: per-client sizes (first 12): [2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560]
  Total samples this round: 122880

Round 2: per-client sizes (first 12): [2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560]
  Total samples this round: 122880

Round 3: per-client sizes (first 12): [2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560]
  Total samples this round: 122880

Round 4: per-client sizes (first 12): [2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560]
  Total samples this round: 122880

Round 5: per-client sizes (first 12): [2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560, 2560]
  Total samples this round: 122880



In [238]:
for i, batch in enumerate(Dataloaders['Test']):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

Batch 1 size: 64
Batch 2 size: 64
Batch 3 size: 64
Batch 4 size: 64
Batch 5 size: 64
Batch 6 size: 64
Batch 7 size: 64
Batch 8 size: 64
Batch 9 size: 64
Batch 10 size: 64
Batch 11 size: 64
Batch 12 size: 64
Batch 13 size: 64
Batch 14 size: 64
Batch 15 size: 64
Batch 16 size: 64
Batch 17 size: 64
Batch 18 size: 64
Batch 19 size: 64
Batch 20 size: 64
Batch 21 size: 64
Batch 22 size: 64
Batch 23 size: 64
Batch 24 size: 64
Batch 25 size: 64
Batch 26 size: 64
Batch 27 size: 64
Batch 28 size: 64
Batch 29 size: 64
Batch 30 size: 64
Batch 31 size: 64
Batch 32 size: 64
Batch 33 size: 64
Batch 34 size: 64
Batch 35 size: 64
Batch 36 size: 64
Batch 37 size: 64
Batch 38 size: 64
Batch 39 size: 64
Batch 40 size: 64
Batch 41 size: 64
Batch 42 size: 64
Batch 43 size: 64
Batch 44 size: 64
Batch 45 size: 64
Batch 46 size: 64
Batch 47 size: 64
Batch 48 size: 64
Batch 49 size: 64
Batch 50 size: 64
Batch 51 size: 64
Batch 52 size: 64
Batch 53 size: 64
Batch 54 size: 64
Batch 55 size: 64
Batch 56 size: 64
B

In [239]:
for i, batch in enumerate(Dataloaders[5][0]):
    batch_size = len(batch[0])  # Assuming the first element of the batch is the data
    print(f"Batch {i+1} size: {batch_size}")
    if batch_size != 64:
        print(f"Batch {i+1} does not contain 64 records.")
        break

Batch 1 size: 64
Batch 2 size: 64
Batch 3 size: 64
Batch 4 size: 64
Batch 5 size: 64
Batch 6 size: 64
Batch 7 size: 64
Batch 8 size: 64
Batch 9 size: 64
Batch 10 size: 64
Batch 11 size: 64
Batch 12 size: 64
Batch 13 size: 64
Batch 14 size: 64
Batch 15 size: 64
Batch 16 size: 64
Batch 17 size: 64
Batch 18 size: 64
Batch 19 size: 64
Batch 20 size: 64
Batch 21 size: 64
Batch 22 size: 64
Batch 23 size: 64
Batch 24 size: 64
Batch 25 size: 64
Batch 26 size: 64
Batch 27 size: 64
Batch 28 size: 64
Batch 29 size: 64
Batch 30 size: 64
Batch 31 size: 64
Batch 32 size: 64
Batch 33 size: 64
Batch 34 size: 64
Batch 35 size: 64
Batch 36 size: 64
Batch 37 size: 64
Batch 38 size: 64
Batch 39 size: 64
Batch 40 size: 64


In [240]:
from collections import Counter
for CLUSTER in range (1, 9):
    DEVICE_PERCENTAGE = []
    for DEVICE__ in range(0,NUM_CLIENTS):
        for i, batch in enumerate(Dataloaders[CLUSTER][DEVICE__]):
            _, labels = batch
            class_counts = Counter(labels.numpy())
            total_records = sum(class_counts.values())
            class_0_count = class_counts.get(0, 0)
            percentage_class_0 = (class_0_count / total_records) * 100
            DEVICE_PERCENTAGE.append(percentage_class_0)
            # print(f"Batch {i+1}: {dict(class_counts)}")
            # print(f"Percentage of class 0: {percentage_class_0:.2f}%\n")
    # print(DEVICE_PERCENTAGE)        
    chunk_size = 6
    averages = [sum(DEVICE_PERCENTAGE[i:i + chunk_size]) / chunk_size for i in range(0, len(DEVICE_PERCENTAGE), chunk_size)]
    # print("Averages of every device:")
    # print(averages)
    chunk_size_4 = 4
    averages = [sum(averages[i:i + chunk_size_4]) / chunk_size_4 for i in range(0, len(averages), chunk_size_4)]
    # print("Averages of every 4 devices:")
    print(averages)

[30.013020833333332, 100.0, 71.80989583333334, 50.0, 28.971354166666668, 100.0, 70.18229166666666, 52.083333333333336, 28.450520833333336, 100.0, 69.46614583333334, 50.390625, 30.208333333333332, 100.0, 72.4609375, 50.846354166666664, 30.2734375, 100.0, 68.1640625, 50.5859375, 29.1015625, 100.0, 69.01041666666667, 50.1953125, 28.971354166666668, 100.0, 70.5078125, 50.06510416666667, 30.143229166666668, 100.0, 68.61979166666667, 51.30208333333333, 31.25, 100.0, 68.03385416666667, 48.502604166666664, 30.078125, 100.0, 69.79166666666666, 52.1484375, 30.338541666666668, 100.0, 67.96875, 51.302083333333336, 29.752604166666668, 100.0, 70.96354166666667, 49.8046875, 29.1015625, 100.0, 69.7265625, 49.088541666666664, 30.533854166666668, 100.0, 69.140625, 50.9765625, 30.598958333333332, 100.0, 67.70833333333333, 49.73958333333333, 28.255208333333336, 100.0, 68.09895833333333, 49.21875, 30.533854166666668, 100.0, 68.09895833333333, 51.04166666666667, 29.361979166666664, 100.0, 71.54947916666667,

In [241]:
del TrafficData

<font color='Red'>***Neural Network***</font>
---
--- 

In [242]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()        
        self.layer_1 = nn.Linear(98, 64)
        self.layer_2 = nn.Linear(64, 32)
        self.layer_out = nn.Linear(32, 15) 
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.relu(x)
        x = self.layer_out(x)        
        return x

In [243]:
# Random  = Net()
# for param_tensor in Random.state_dict():
#     print(param_tensor, "\t", Random.state_dict()[param_tensor].size())
# torch.save(Random.state_dict(), "0_Input_Random_model_Net.pth")

In [244]:
def train(net, trainloader, epochs: int, verbose=True):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    net.train()
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
            predictions = torch.max(outputs.data, 1)[1]
            prediction_matrix.append(predictions.tolist())
            actual_matrix.append(labels.tolist())
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        loss_matrix.append(epoch_loss.tolist())
        acc_matrix.append(epoch_acc)
    return prediction_matrix, actual_matrix, acc_matrix, loss_matrix
def test(net, testloader):
    criterion = nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())
    loss /= len(testloader.dataset)
    accuracy = correct / total
    loss_matrix.append(loss)
    acc_matrix.append(accuracy)    
    print(f"Evaluation: eval loss {loss}, eval accuracy {accuracy}")
    return loss, accuracy, prediction_matrix, actual_matrix, acc_matrix, loss_matrix

In [245]:
prediction_dict= {}
actual_dict= {}
accuracy_dict= {}
loss_dict= {}

<font color='Brown'>***Federated Learning Classes***</font>
---
--- 

In [246]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [247]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, FL_Update):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.FL_Update = FL_Update
    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)
    def fit(self, parameters, config):
        local_epochs = config["local_epochs"]
        print(f"[Client {self.cid}, round {self.FL_Update}] fit, config: {config}")
        set_parameters(self.net, parameters)
        _1, _2, _3, _4 = train(self.net, self.trainloader, epochs=local_epochs)
        prediction_dict[f'C{self.cid}R{self.FL_Update}'] = _1
        actual_dict[f'C{self.cid}R{self.FL_Update}'] = _2
        accuracy_dict[f'C{self.cid}R{self.FL_Update}'] = _3
        loss_dict[f'C{self.cid}R{self.FL_Update}'] = _4
        # Save model updates (parameters)
        # update_filename = f'EdgeCooperation/Performance/Results/C{self.cid}R{self.FL_Update}_update.pkl'
        # with open(update_filename, 'wb') as update_outfile:
        #     pickle.dump(get_parameters(self.net), update_outfile)        

        return get_parameters(self.net), len(self.trainloader), {}
    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}")
        print(f"[Client {self.cid}] evaluate, config: {config}")
        return None

In [248]:
def retrieve_and_sort_client_updates(global_model, round_number, client_ids):
    client_updates = {}
    for cid in client_ids:
        update_filename = f'EdgeCooperation/Performance/Results/C{cid}R{round_number}_update.pkl'
        with open(update_filename, 'rb') as update_file:
            client_update = pickle.load(update_file)
            client_updates[cid] = client_update
    client_contributions = {cid: calculate_weight_magnitude(global_model, update) for cid, update in client_updates.items()}
    sorted_clients = sorted(client_contributions.items(), key=lambda x: x[1], reverse=True)
    least_contributing_clients = sorted_clients[-3:]
    return sorted_clients, least_contributing_clients

def calculate_weight_magnitude(global_model, client_update):
    """
    Calculate the L2 norm of the weight difference between the global model and client's updated model.
    
    Args:
    global_model (nn.Module): The global model before client update.
    client_update (list): List of numpy arrays representing client's updated model parameters.

    Returns:
    float: The L2 norm of the weight difference.
    """
    weight_diff = 0.0
    global_parameters = [param.detach().cpu().numpy() for param in global_model.parameters()]
    for global_param, client_param in zip(global_parameters, client_update):
        weight_diff += np.linalg.norm(global_param - client_param)
    return weight_diff


<font color='Brown'>***Clients Functions***</font>
---
--- 

In [249]:
def General_Client():
    def client_fn(cid: int, Round: int) -> FlowerClient:
        clients_ids_list = TrainingListPerRound[Round]
        if int(cid) in clients_ids_list:
            net = Net().to(DEVICE)
            trainloader = Dataloaders[Round][int(cid)]
            arg_ = Round
            return FlowerClient(cid, net, trainloader, arg_)
        else:
            raise ValueError(f"Client ID {cid} not found in the list for round {Round}")
    return client_fn

<font color='Brown'>***FL Strategy***</font>
---
--- 

In [250]:
Global_Models = {}
class SaveModelStrategy(fl.server.strategy.FedAvg):
    def __init__(self, additional_argument, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.additional_argument = additional_argument
    def aggregate_fit(self, rnd: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitRes]], failures: List[BaseException]) -> Optional[fl.common.NDArrays]:
        aggregated_parameters_tuple = super().aggregate_fit(rnd, results, failures)
        aggregated_parameters, _ = aggregated_parameters_tuple
        if aggregated_parameters is not None:
            # print(f"Saving round {rnd} aggregated_parameters...")
            # Convert `Parameters` to `List[np.ndarray]`
            aggregated_weights: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_parameters)
            # Convert `List[np.ndarray]` to PyTorch`state_dict`
            Global_Models[self.additional_argument] = Net()
            params_dict = zip(Global_Models[self.additional_argument].state_dict().keys(), aggregated_weights)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            Global_Models[self.additional_argument].load_state_dict(state_dict, strict=True)
            torch.save(Global_Models[self.additional_argument].state_dict(), f"{PATH}/GlobalModel_{self.additional_argument}.pth")
        return aggregated_parameters_tuple

In [251]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.
    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "current_round": server_round,  # The current round of federated learning
        "local_epochs": EPOCHS #1 if rnd < 2 else 2,  # 
    }
    return config

***Running Centralized DNN Rounds***
---
---

In [252]:
# ============================================================
# Centralized Round-by-Round Training (No FL)
# ============================================================

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import ConcatDataset, DataLoader


# ============================================================
# Training / Evaluation
# ============================================================

def train_one_round(model, trainloader, epochs):
    """Runs EPOCHS of training on this round's dataset."""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    model.train()

    for epoch in range(epochs):
        correct, total, running_loss = 0, 0, 0.0
        for Xbatch, ybatch in trainloader:
            Xbatch, ybatch = Xbatch.to(DEVICE), ybatch.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(Xbatch)
            loss = criterion(outputs, ybatch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            total += ybatch.size(0)
            correct += (torch.max(outputs, 1)[1] == ybatch).sum().item()

        print(f"  Epoch {epoch+1}/{epochs} | "
              f"Loss: {running_loss/len(trainloader):.6f} | "
              f"Acc: {correct/total:.4f}")


def test_model(model, testloader):
    """Runs a simple test pass (no saving)."""
    criterion = nn.CrossEntropyLoss()
    model.eval()
    correct, total, running_loss = 0, 0, 0.0

    with torch.no_grad():
        for Xbatch, ybatch in testloader:
            Xbatch, ybatch = Xbatch.to(DEVICE), ybatch.to(DEVICE)
            outputs = model(Xbatch)
            loss = criterion(outputs, ybatch).item()

            running_loss += loss
            total += ybatch.size(0)
            correct += (torch.max(outputs, 1)[1] == ybatch).sum().item()

    print(f"  Test → Loss: {running_loss/len(testloader):.6f}, "
          f"Acc: {correct/total:.4f}")


# ============================================================
# Centralized Training Loop
# ============================================================
if not FL:
    os.makedirs(PATH, exist_ok=True)
    print("🔥 Starting centralized Round-by-Round training...\n")

    model = Net().to(DEVICE)

    for Round in range(1, ROUNDS + 1):

        print("========================")
        print(f"   ROUND {Round} START")
        print("========================")

        # Build round dataset from client partitions
        client_datasets = [
            loader.dataset for loader in Dataloaders[Round]
            if hasattr(loader, "dataset") and len(loader.dataset) > 0
        ]

        round_dataset = ConcatDataset(client_datasets)
        trainloader = DataLoader(round_dataset, batch_size=64, shuffle=True)

        print(f"  Train samples this round: {len(round_dataset)}")

        # Train
        train_one_round(model, trainloader, EPOCHS)

        # Evaluate (optional)
        print(f"  Evaluating round {Round} model...")
        test_model(model, Dataloaders["Test"])

        # Save model
        save_path = f"{PATH}/GlobalModel_{Round}.pth"
        torch.save(model.state_dict(), save_path)
        print(f"  💾 Saved: {save_path}")

        print("========================")
        print(f"   ROUND {Round} DONE")
        print("========================\n")

    print("🏁 Centralized Round-by-Round training completed.")


***Running the Generalized FL DNN Round***
---
---

In [ ]:
if FL:
    os.makedirs(PATH, exist_ok=True)
    print("Loading Initial Global Model")
    Global_Models[0] = Net()
    Global_Models[0].load_state_dict(torch.load("0_Input_Random_model_Net.pth"))
    Global_Models[0].train()

    TrainingListPerRound = {}
    for Round in range(1, ROUNDS+1):
        TrainingListPerRound[Round] = []     
        for CLIENT in range (NUM_CLIENTS):
            TrainingListPerRound[Round].append(int(CLIENT))

    for Round in range(1, ROUNDS+1):
        print("Starting FL Round: ", Round)
        strategy = SaveModelStrategy(
                fraction_fit=1.0,  # Sample 100% of available clients for training
                fraction_evaluate=0,  # Sample 50% of available clients for evaluation
                min_fit_clients=2,  # Never sample less than 10 clients for training
                min_evaluate_clients=0,  # Never sample less than 5 clients for evaluation
                min_available_clients=2,  # Wait until all 10 clients are available
                on_fit_config_fn=fit_config,
                initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Global_Models[Round-1])),
                additional_argument = Round
        )
        # print(f'Current training nodes at round {Round}: ', len(TrainingListPerRound[Round]), ' ', TrainingListPerRound[Round])
        client_fn = General_Client()
        fl.simulation.start_simulation(
            client_fn=lambda cid: client_fn(cid, int(Round)),
            num_clients=int(NUM_CLIENTS),
            config=fl.server.ServerConfig(num_rounds=int(1)),
            client_resources={"num_cpus":16, "num_gpus":1}, 
            ray_init_args = {'num_cpus': 16, 'num_gpus': 1},
            strategy=strategy
        )
        print("End of FL Round: ", Round)
        print("Loading Global Model: ", Round)
        Global_Models[Round] = Net()
        Global_Models[Round].load_state_dict(torch.load(f"{PATH}/GlobalModel_{Round}.pth"))
        Global_Models[Round].train()

Loading Initial Global Model
Starting FL Round:  1


INFO flwr 2026-01-06 20:25:19,029 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)


2026-01-06 20:25:22,835	INFO worker.py:2012 -- Started a local Ray instance.
INFO flwr 2026-01-06 20:25:27,710 | app.py:210 | Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'GPU': 1.0, 'CPU': 16.0, 'memory': 7823568077.0, 'object_store_memory': 3352957747.0, 'node:10.192.12.245': 1.0}
INFO flwr 2026-01-06 20:25:27,711 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 16, 'num_gpus': 1}
INFO flwr 2026-01-06 20:25:27,724 | app.py:270 | Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO flwr 2026-01-06 20:25:27,725 | server.py:89 | Initializing global parameters
INFO flwr 2026-01-06 20:25:27,726 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2026-01-06 20:25:27,727 | server.py:91 | Evaluating initial parameters
INFO flwr 2026-01-06 20:25:27,728 | server.py:104 | FL starting
DEBUG flwr 2026-01-06 20:25:27,728 | server.py:222 | fit_round 1: strategy sampled 48 clients (out of 48)


(DefaultActor pid=23406) [Client 37, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23406) [Client 34, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23406) [Client 17, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23406) [Client 3, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23406) [Client 25, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23406) [Client 16, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23406) [Client 22, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23406) [Client 8, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23406) [Client 38, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23406) [Client 42, round 1] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActo

(pid=gcs_server) [2026-01-06 20:25:50,592 E 23288 23288] (gcs_server) gcs_server.cc:302: Failed to establish connection to the event+metrics exporter agent. Events and metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


...

(DefaultActor pid=23958) [Client 4, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}


(raylet) [2026-01-06 20:26:57,619 E 23907 23907] (raylet) main.cc:975: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


(DefaultActor pid=23958) [Client 6, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23958) [Client 17, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23958) [Client 46, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23958) [Client 29, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}


(DefaultActor pid=23958) [2026-01-06 20:27:02,179 E 23958 24023] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14
[2026-01-06 20:27:02,569 E 15062 23956] core_worker_process.cc:825: Failed to establish connection to the metrics exporter agent. Metrics will not be exported. Exporter agent status: RpcError: Running out of retries to initialize the metrics agent. rpc_code: 14


(DefaultActor pid=23958) [Client 32, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23958) [Client 31, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23958) [Client 45, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23958) [Client 15, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23958) [Client 24, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23958) [Client 42, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23958) [Client 47, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}
(DefaultActor pid=23958) [Client 26, round 2] fit, config: {'current_round': 1, 'local_epochs': 3}


<font color='Grey'>***Performance Testing***</font>
---
--- 

In [ ]:
import os
import glob

# Define the directory and file pattern
directory = PATH + '/'
pattern = "GlobalModel_*.pth"

# Find all matching files
files = glob.glob(os.path.join(directory, pattern))

# Extract numbers from file names
numbers = []
for file in files:
    base_name = os.path.basename(file)
    num_str = base_name.replace("GlobalModel_", "").replace(".pth", "")
    try:
        numbers.append(int(num_str))
    except ValueError:
        pass

# Determine the maximum number
max_num = max(numbers) if numbers else 0
print(max_num)

# Use the max_num in a loop
for num in range(1, max_num + 1):
    file_path = f"{PATH}/GlobalModel_{num}.pth"
    if os.path.exists(file_path):
        # Load the file or perform any operation you need
        print(f"Loading {file_path}")
    else:
        print(f"File {file_path} does not exist")

40
Loading DNNCent11atk-40-rounds-3-epochs-0.0025-lr-1920-DG-40-batch-2560-size/GlobalModel_1.pth
Loading DNNCent11atk-40-rounds-3-epochs-0.0025-lr-1920-DG-40-batch-2560-size/GlobalModel_2.pth
Loading DNNCent11atk-40-rounds-3-epochs-0.0025-lr-1920-DG-40-batch-2560-size/GlobalModel_3.pth
Loading DNNCent11atk-40-rounds-3-epochs-0.0025-lr-1920-DG-40-batch-2560-size/GlobalModel_4.pth
Loading DNNCent11atk-40-rounds-3-epochs-0.0025-lr-1920-DG-40-batch-2560-size/GlobalModel_5.pth
Loading DNNCent11atk-40-rounds-3-epochs-0.0025-lr-1920-DG-40-batch-2560-size/GlobalModel_6.pth
Loading DNNCent11atk-40-rounds-3-epochs-0.0025-lr-1920-DG-40-batch-2560-size/GlobalModel_7.pth
Loading DNNCent11atk-40-rounds-3-epochs-0.0025-lr-1920-DG-40-batch-2560-size/GlobalModel_8.pth
Loading DNNCent11atk-40-rounds-3-epochs-0.0025-lr-1920-DG-40-batch-2560-size/GlobalModel_9.pth
Loading DNNCent11atk-40-rounds-3-epochs-0.0025-lr-1920-DG-40-batch-2560-size/GlobalModel_10.pth
Loading DNNCent11atk-40-rounds-3-epochs-0.0025

In [ ]:
def view_pickle(path, G=None, suffix=None):
    print(G)
    if G is not None:
        rounds = [G]
    else:
        # Auto-detect how many Global_X files exist
        rounds = sorted({
            int(f.split("_")[1])
            for f in os.listdir(path)
            if f.startswith("Global_") and f.split("_")[1].isdigit()
        })

    suffixes = [suffix] if suffix else ["pred", "actual", "accurracy", "loss"]

    for g in rounds:
        for s in suffixes:
            filename = os.path.join(path, f"Global_{g}_{s}")
            if not os.path.exists(filename):
                print(f"Missing file: {filename}")
                continue
            try:
                with open(filename, "rb") as f:
                    data = pickle.load(f)
                print(f"File: {filename}")
                if isinstance(data, list) and len(data) > 2:
                    pprint(data[:2])
                    print(f"... ({len(data)} total items)")
                else:
                    pprint(data)
            except Exception as e:
                print(f"❌ Error reading {filename}: {e}")



In [ ]:
pred_test = {}
actual_test = {}
accuracy_test = {}
loss_test = {}
G = 0

for num in range(1, max_num+1):
    model = Net()
    model.load_state_dict(torch.load(f"{PATH}/GlobalModel_{num}.pth"))
    model.eval()
    
    prediction_matrix = []
    actual_matrix= []
    acc_matrix = []
    loss_matrix=[]
    G = G + 1
    criterion = nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    with torch.no_grad():
        for images, labels in Dataloaders['Test']:
            outputs = model(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            prediction_matrix.append(predicted.tolist())
            actual_matrix.append(labels.tolist())
    loss /= len(Dataloaders['Test'].dataset)
    accuracy = correct / total
    loss_matrix.append(loss)
    acc_matrix.append(accuracy) 

    pred_test[f'Global_{G}'] = prediction_matrix
    actual_test[f'Global_{G}'] = actual_matrix
    accuracy_test[f'Global_{G}'] = acc_matrix
    loss_test[f'Global_{G}'] = loss_matrix 

    filename = f'{PATH}/Global_{G}_pred'
    outfile = open(filename,'wb')
    pickle.dump(pred_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_actual'
    outfile = open(filename,'wb')
    pickle.dump(actual_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_accurracy'
    outfile = open(filename,'wb')
    pickle.dump(accuracy_test[f'Global_{G}'],outfile)
    outfile.close()

    filename = f'{PATH}/Global_{G}_loss'
    outfile = open(filename,'wb')
    pickle.dump(loss_test[f'Global_{G}'],outfile)
    outfile.close()